In [ ]:
import numpy as np
import pickle as pkl
from database_tools.processing.cardiac import estimate_pulse_rate, estimate_spo2
from database_tools.processing.detect import detect_peaks
from database_tools.tools.dataset import ConfigMapper
from gcp_utils.tools.preprocess import _clean_frame, _split_frame
from plotly.subplots import make_subplots

def clean_peaks_spo2(sig, idx, thresh):
    peaks, troughs = idx.values()
    peak_vals = sig[peaks]
    trough_vals = sig[troughs]

    mean = np.mean(peak_vals)
    mask = np.where( (peak_vals < (mean + thresh)) & (peak_vals > (mean - thresh)) )
    peaks_cleaned = peaks[mask]

    mean = np.mean(trough_vals)
    mask = np.where( (trough_vals < (mean + thresh)) & (trough_vals > (mean - thresh)) )
    troughs_cleaned = troughs[mask]
    return dict(peaks=peaks_cleaned, troughs=troughs_cleaned)

def predict_cardiac_metrics(red: list, ir: list, cm: ConfigMapper) -> dict:
    i = 1
    red = _split_frame(red, 4, dtype='ndarray')[i]
    ir = _split_frame(ir, 4, dtype='ndarray')[i]

    red_idx = detect_peaks(np.array(red))
    ir_idx = detect_peaks(np.array(ir))

    red_idx = clean_peaks_spo2(red, red_idx, 1000)
    ir_idx = clean_peaks_spo2(ir, ir_idx, 1000)
    # return red_idx, ir_idx

    pulse_rate = estimate_pulse_rate(red, ir, red_idx, ir_idx, fs=cm.deploy.bpm_fs)
    spo2, r = estimate_spo2(red, ir, red_idx, ir_idx)
    result = {
        'pulse_rate': int(pulse_rate),
        'spo2': float(spo2),
        'r': float(r)
    }
    return result

In [ ]:
with open('bpm-frame.pkl', 'rb') as f:
    data = pkl.load(f)

red_frame = data['red_frame']
ir_frame = data['ir_frame']

cm = ConfigMapper('/home/cam/Documents/gcp_utils/gcp_utils/config.ini')

In [ ]:
thresh = 2000

red_clean = _clean_frame(np.array(red_frame), thresh=thresh)
ir_clean = _clean_frame(np.array(ir_frame), thresh=thresh)

In [ ]:
predict_cardiac_metrics(red_clean, ir_clean, cm=cm)

In [ ]:
red_idx, ir_idx = predict_cardiac_metrics(red_clean, ir_clean, cm=cm)
red_peaks, red_troughs = red_idx.values()
ir_peaks, ir_troughs = ir_idx.values()

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(y=red_clean, row=1, col=1, name='red')
fig.add_scatter(x=red_peaks, y=red_clean[red_peaks], row=1, col=1, mode='markers', name='red')
fig.add_scatter(x=red_troughs, y=red_clean[red_troughs], row=1, col=1, mode='markers', name='red')

fig.add_scatter(y=ir_clean, row=2, col=1, name='red')
fig.add_scatter(x=ir_peaks, y=ir_clean[ir_peaks], row=2, col=1, mode='markers', name='ir')
fig.add_scatter(x=ir_troughs, y=ir_clean[ir_troughs], row=2, col=1, mode='markers', name='ir')